# ABC Guide for XMM-Newton -- Chapter 10 (RGS Data Processing)

---

#### Introduction
This tutorial is based on Chapter 10 from the [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide") prepared by the NASA/GSFC XMM-Newton Guest Observer Facility. This notebook assumes you are using the version of pySAS found on [GitHub](https://github.com/XMMGOF/pysas) and have already configured it to work with your SAS installation (see the [README on GitHub](https://github.com/XMMGOF/pysas/blob/main/README.md)). 
#### Expected Outcome
The ability to process RGS data and prepare it for analysis.
#### SAS Tasks to be Used

- `rgsproc`[(Documentation for epproc)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/rgsproc/index.html "rgsproc Documentation")
- `evselect`[(Documentation for evselect)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/evselect/index.html)
- `tabgtigen`[(Documentation for tabgtigen)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/tabgtigen/index.html)
- `gtibuild`[(Documentation for gtibuild)](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/gtibuild/index.html)

#### Prerequisites
<div class="alert alert-block alert-info">
    <b>Note:</b> Before running this notebook, or even starting a Jupyter Lab session, HEASOFT has to be initialized. If you did not initalize HEASOFT before starting this Jupyter Lab session, or opening this notebook, please close this window and initalize HEASOFT (it is not possible to initalize HEASOFT from within a Jupyter Notebook). SAS defaults for your machine will need to be set as explained in the README on GitHub (https://github.com/XMMGOF/pysas/blob/main/README.md).
</div>

#### Useful Links

- [`pysas` Documentation](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/pysas/index.html "pysas Documentation")
- [`pysas` on GitHub](https://github.com/XMMGOF/pysas)
- [Common SAS Threads](https://www.cosmos.esa.int/web/xmm-newton/sas-threads "SAS Threads")
- [Users' Guide to the XMM-Newton Science Analysis System (SAS)](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/sas_usg/USG/SASUSG.html "Users' Guide")
- [The XMM-Newton ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/ "ABC Guide")
- [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html "Helpdesk") - Link to form to contact the GOF Helpdesk.

#### Caveats
This tutorial uses an observation of Mkn 421 (obsid = '0153950701').

##### Last Reviewed: _25 June 2024, for SAS v21_
##### Last Updated: _25 June 2024_
##### By: Ryan Tanner (ryan.tanner@nasa.gov)
---

In [ ]:
# pySAS imports
import pysas
from pysas.wrapper import Wrapper as w

# Useful imports
import os
import subprocess

# Imports for plotting
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
plt.style.use(astropy_mpl_style)

### 10.1 : Rerun basic processing

<div class="alert alert-block alert-info">
    <b>Note:</b> Running rgsproc on this particular obsid will take A LONG TIME, depending on your machine. Be prepared to wait.
</div>

In [ ]:
obsid = '0153950701'
odf = pysas.odfcontrol.ODFobject(obsid)

We start by reprocessing the data. Since we are only interested in the RGS data we do not have to run `epproc` and `emproc`. By default `basic_setup` will run `epproc`,`emproc`, and `rgsproc`, so we will set `run_epproc` and `run_emproc` to `False`.

We will also use the rgsproc inputs, `orders='1 2' bkgcorrect=no withmlambdacolumn=yes spectrumbinning=lambda`.

In [ ]:
rgsproc_args = ["orders='1 2'",
                'bkgcorrect=no',
                'withmlambdacolumn=yes',
                'spectrumbinning=lambda']
odf.basic_setup(overwrite=False,repo='heasarc',rerun=True,run_epproc=False,run_emproc=False,rgsproc_args=rgsproc_args)

The input arguments for `rgsproc` are:

    orders - dispersion orders to extract
    bkgcorrect - subtract background from source spectra?
    withmlambdacolumn - include a wavelength column in the event file product
    spectrumbinning - accumulate the spectrum either in wavelength or beta space

Note the last keyword, `spectrumbinning`. If you want to merge data from the same orders in RGS1 and RGS2, keep it at the default value `lambda`. If you want to merge data from the same instrument, with different orders, set it to `beta`. Merging spectra is discussed in §10.6.

This takes several minutes, and outputs 12 files per RGS, plus 3 general use FITS files. As before, links to the event list files are stored in `odf.files['R1evt_list']` and `odf.files['R2evt_list']`.

In [ ]:
print(odf.files['R1evt_list'])
print(odf.files['R2evt_list'])

In [ ]:
odf.get_event_lists()

### 10.1.1 : Potentially useful tips for using the pipeline

The pipeline task, rgsproc, is very flexible and can address potential pitfalls for RGS users. In §10.1, we used a simple set of parameters with the task; if this is sufficient for your data (and it should be for most), feel free to skip to later sections, where data filters are discussed. In the following subsections, we will look at the cases of a nearby bright optical source, a nearby bright X-ray source, and a user-defined source.

### 10.1.2 : A Nearby Bright Optical Source

With certain pointing angles, zeroth-order optical light may be reflected off the telescope optics and cast onto the RGS CCD detectors. If this falls on an extraction region, the current energy calibration will require a wavelength-dependent zero-offset. Stray light can be detected on RGS DIAGNOSTIC images taken before, during and after the observation. This test, and the offset correction, are not performed on the data before delivery. Please note that this will not work in every case. If a source is very bright, the diagnostic data that this relies on may not have been downloaded from the telescope in order to save bandwidth. Also, the RGS target itself cannot be the source of optical photons, as the spectrum's zero-order falls far from the RGS chip array. To check for stray light and apply the appropriate offsets, use the following inputs.

```python
rgsproc_args = ["orders='1 2'",
                'bkgcorrect=no',
                'calcoffsets=yes',
                'withoffsethistogram=no']
```

where the parameters are as described in §10.1 and
    
calcoffsets - calculate PHA offsets from diagnostic image    s
withoffsethistogram - produce a histogram of uncalibrated excess for the user

### 10.1.3 : A Nearby Bright X-ray Source

In the example above, it is assumed that the field around the source contains sky only. Provided a bright background source is well-separated from the target in the cross-dispersion direction, a mask can be created that excludes it from the background region. Here the source has been identified in the EPIC images and its coordinates have been taken from the EPIC source list which is included among the pipeline products. The bright neighboring object is found to be the third source listed in the sources file. The first source is the target. The inputs would be

```python
rgsproc_args = ["orders='1 2'",
                'bkgcorrect=no',
                'withepicset=yes',
                'epicset=P0153950701EPX000OMSRLI0000.FTZ',
                "exclsrcsexpr='INDEX==1&&INDEX==3'"]
```

where the parameters are as described in §10.1 and

    withepicset - calculate extraction regions for the sources contained in an EPIC source list
    epicset - name of the EPIC source list, such as generated by emldetect or eboxdetect procedures
    exclsrcsexpr - expression to identify which source(s) should be excluded from the background extraction region

<div class="alert alert-block alert-warning">
    <b>Notice:</b> This method uses an <b>OMSRLI</b> file which is found in the pipeline products (PPS). <b>OMSRLI</b> stands for Observation Maximum-Likelihood Source List, in this case OM does <i>not</i> stand for 'Optical Monitor'. You will need to download the PPS files for this. This can be done by using the command <b>odf.calibrate_odf(overwrite=False,level='PPS')</b>. The file will be in the '$data_dir/obsid/PPS/' directory.
</div>

### 10.1.4 : User-defined Source Coordinates

If the true coordinates of an object are not included in the EPIC source list or the science proposal, the user can define the coordinates of a new source by typing:

```python
rgsproc_args = ["orders='1 2'",
                'bkgcorrect=no',
                'withsrc=yes',
                'srclabel=Mkn421',
                'srcstyle=radec',
                'srcra=166.113808',
                'srcdec=+38.208833']
```

where the parameters are as described in §10.1 and

    withsrc - make the source be user-defined
    srclabel - source name
    srcstyle - coordinate system in which the source position is defined
    srcra - the source's right ascension in decimal degrees
    srcdec - the source's declination in decimal degrees
    
Since the event files are current, we can proceed with some simple analysis demonstrations, which will allow us to generate filters. Rememer that all tasks should be called from the work directory, and that tasks place output files in whatever directory you are in when they are called.

### 10.2 : Create and Display an Image

Two commonly-made plots are those showing PI vs. BETA_CORR (also known as 'banana plots') and XDSP_CORR vs. BETA_CORR.

The input arguments to `evselect` to create these FITS image files are:

    table - input event table
    withimageset - make an image
    imageset - name of output image
    xcolumn - event column for X axis
    ycolumn - event column for Y axis
    imagebinning - form of binning, force entire image into a given size or bin by a specified number of pixels
    ximagesize - output image pixels in X
    yimagesize - output image pixels in Y

In [ ]:
def make_fits_image(event_list_file, image_file='image.fits', xcolumn='BETA_CORR', ycolumn='PI', expression=None):
    
    inargs = {}
    inargs['table']        = event_list_file+':EVENTS'
    inargs['withimageset'] = 'yes'
    inargs['imageset']     = image_file
    inargs['xcolumn']      = xcolumn
    inargs['ycolumn']      = ycolumn
    inargs['imagebinning'] = 'imageSize'
    inargs['ximagesize']   = '600'
    inargs['yimagesize']   = '600'
    if expression != None:
        inargs['expression'] = expression
    
    w('evselect', inargs).run()

    hdu = fits.open(image_file)[0]
    plt.imshow(hdu.data, origin='lower', norm='log')
    plt.colorbar()
    plt.show()

In [ ]:
os.chdir(odf.work_dir)
R1_event_list = odf.files['R1evt_list'][0]
make_fits_image(R1_event_list,image_file='pi_bc.fits')
make_fits_image(R1_event_list,image_file='xd_bc.fits', xcolumn='BETA_CORR', ycolumn='XDSP_CORR')

### 10.3 : Create and Display a Light Curve

The background is assessed through examination of the light curve. We will extract a region, CCD9, that is most susceptible to proton events and generally records the least source events due to its location close to the optical axis. Also, to avoid confusing solar flares for source variability, a region filter that removes the source from the final event list should be used. The region filters are kept in the source file product `*SRCLI_*.FIT`. `rgsproc` outputs an `M_LAMBDA` column which can be used to generate the light curve. (The `*SRCLI_*.FIT` file that came with the PPS products contains a `BETA_CORR` column if you prefer to use that instead.)

The input arguments to `evselect` to create a light curve file are:

    table - input event table
    withrateset - make a light curve
    rateset - name of output light curve file
    maketimecolumn - control to create a time column
    timecolumn - time column label
    timebinsize - time binning (seconds)
    makeratecolumn - control to create a count rate column, otherwise a count column will be created
    expression - filtering expression


In [ ]:
def plot_light_curve(event_list_file, light_curve_file='ltcrv.fits',expression=None):
                     
    inargs = {'table': event_list_file, 
              'withrateset': 'yes', 
              'rateset': light_curve_file, 
              'maketimecolumn': 'yes', 
              'timecolumn': 'TIME', 
              'timebinsize': '50', 
              'makeratecolumn': 'yes'}

    if expression != None:
        inargs['expression'] = expression

    w('evselect', inargs).run()

    ts = Table.read(light_curve_file,hdu=1)
    plt.plot(ts['TIME'],ts['RATE'])
    plt.xlabel('Time (s)')
    plt.ylabel('Count Rate (ct/s)')
    plt.show()

Sometimes, it is necessary to use filters on time in addition to those mentioned above. This is because of soft proton background flaring, which can have count rates of 100 counts/sec or higher across the entire bandpass.

To determine if our observation is affected by background flaring, we can examine the light curve. For the time binning, we will set it to something reasonable (usually between 10 and 100 s).

In [ ]:
expression = '(CCDNR==9)&&(REGION(P0153950701R1S001SRCLI_0000.FIT:RGS1_BACKGROUND,M_LAMBDA,XDSP_CORR))'
plot_light_curve(R1_event_list, light_curve_file='r1_ltcrv.fits', expression=expression)

In this case no flares are evident, so we will continue to the next section. However, if a dataset does contain flares, they should be removed in the same way as shown for EPIC Imaging mode data in §6.5.

### 8.5 : Extract the Source and Background Spectra

The first step in extracting a spectrum from PN Timing data is to make an image of the event file over the energy range we are interested in; for this example, we'll say 0.5-15 keV. And since this is the PN, we need to remember to set `(FLAG==0)` to get a high-quality spectrum. Thus, our expression parameter would be set to `(FLAG==0) && (PI in [500:15000])`, and we make a new image using this expression.

In [ ]:
make_fits_image(basic_filter_file, image_file=final_filter_image, 
                expression="'(FLAG==0) && (PI in [500:15000])'")

The source is centered on `RAWX=37`; we will extract this and the 10 pixels on either side of it:

In [ ]:
expression = "'(FLAG==0) && (PI in [500:15000]) && (RAWX in [27:47])'"

In [ ]:
inargs = {}
inargs['table']           = basic_filter_file
inargs['spectrumset']     = source_pi_file
inargs['energycolumn']    = 'PI'
inargs['spectralbinsize'] = '5'
inargs['specchannelmin']  = '0'
inargs['specchannelmax']  = '20479'
inargs['withfilteredset'] = 'yes'
inargs['filteredset']     = pn_spectra_file
inargs['expression']      = expression

w('evselect', inargs).run()

For the background, the extraction area should be as far from the source as possible. However, sources with > 200 ct/s (like our example!) are so bright that they dominate the entire CCD area, and there is no source-free region from which to extract a background. (It goes without saying that this is highly energy-dependent.) In such a case, it may be best not to subtract a background. Users are referred to Ng et al. (2010, A&A, 522, 96) for an in-depth discussion. While this observation is too bright to have a good background extraction region, the process is shown below nonetheless for the sake of demonstration:

In [ ]:
expression = "'(FLAG==0) && (PI in [500:15000]) && (RAWX in [3:5])'"

In [ ]:
inargs = {}
inargs['table']           = basic_filter_file
inargs['withspectrumset'] = 'yes'
inargs['spectrumset']     = bkg_pi_file
inargs['energycolumn']    = 'PI'
inargs['spectralbinsize'] = '5'
inargs['withspecranges']  = 'yes'
inargs['specchannelmin']  = '0'
inargs['specchannelmax']  = '20479'
inargs['withfilteredset'] = 'yes'
inargs['filteredset']     = pn_bkg_file
inargs['expression']      = expression

w('evselect', inargs).run()

### 8.6 : Check for Pile Up

Depending on how bright the source is and what modes the EPIC detectors are in, event pile up may be a problem. Pile up occurs when a source is so bright that incoming X-rays strike two neighboring pixels or the same pixel in the CCD more than once in a read-out cycle. In such cases the energies of the two events are in effect added together to form one event. If this happens sufficiently often, 

    1. The spectrum will appear to be harder than it actually is, and 
    2. The count rate will be underestimated, since multiple events will be undercounted. 

Briefly, we deal with it in PN Timing data essentially the same way as in Imaging data, that is, by using only single pixel events, and/or removing the regions with very high count rates, checking the amount of pile up, and repeating until it is no longer a problem. We recommend to always check for it.

Note that this procedure requires as input the event files created when the spectrum was made (i.e. `pn_spectra_file = 'pn_filt_source_WithBore.fits'`), not the usual time-filtered event file.

In [ ]:
inargs = ['set={0}'.format(pn_spectra_file),
          'plotfile=pn_epat.ps',
          'useplotfile=yes',
          'withbackgroundset=yes',
          'backgroundset={0}'.format(pn_bkg_file)]

w('epatplot', inargs).run()

In [ ]:
gv_out = subprocess.run(['gv','pn_epat.ps'],stdout = subprocess.DEVNULL)
# Note: You will need to close the window that opens to run the remaining cells in this notebook.

The output of epatplot is a postscript file, `pn_epat.ps`, which may be viewed with viewers such as `gv`, containing two graphs describing the distribution of counts as a function of PI channel; see figure below.

<center><img src="__images/pile_up_Cen_X-3.png"/></center>

A few words about interpretting the plots are in order. The top is the distribution of counts versus PI channel for each pattern class (single, double, triple, quadruple), and the bottom is the expected pattern distribution (smooth lines) plotted over the observed distribution (line with noise). The lower plot shows the model distributions for single and double events and the observed distributions. It also gives the ratio of observed-to-modeled events with $1-\sigma$ uncertainties for single and double pattern events over a given energy range. (The default is 0.5-2.0 keV; this can be changed with the `pileupnumberenergyrange` parameter.) If the data is not piled up, there will be good agreement between the modeled and observed single and double event pattern distributions. Also, the observed-to-modeled fractions for both singles and doubles in the 0.5-2.0 keV range will be unity, within errors. In contrast, if the data is piled up, there will be clear divergence between the modeled and observed pattern distributions, and the observed-to-modeled fraction for singles will be less than 1.0, and for doubles, it will be greater than 1.0.

Finally, when examining the plots, it should noted that the observed-to-modeled fractions can be inaccurate. Therefore, the agreement between the modeled and observed single and double event pattern distributions should be the main factor in determining if an observation is affected by pile up or not.

Examining the plots, we see that there is a large difference between the modeled and observed single and double pattern events at $> 1.0$ keV, but this divergence is not reflected in the observed-to-model fractions since for singles it is $> 1.0$ with $1.011\pm 0.001$, and for doubles it is $<1.0$ with $0.977\pm 0.001$.

To capture the pile up we need to extend the energy range for the observed-to-model fraction calculations. The default is $500-2000$ eV. Let us set the range to $1000-5000$ eV.

In [ ]:
inargs = ['set={0}'.format(pn_spectra_file),
          'plotfile=pn_epat.ps',
          'useplotfile=yes',
          'pileupnumberenergyrange=1000 5000',
          'withbackgroundset=yes',
          'backgroundset={0}'.format(pn_bkg_file)]

w('epatplot', inargs).run()

In [ ]:
gv_out = subprocess.run(['gv','pn_epat.ps'],stdout = subprocess.DEVNULL)
# Note: You will need to close the window that opens to run the remaining cells in this notebook.

Now the cacluated observed-to-model fractions are $0.988\pm 0.001$ for singles, and $1.121\pm 0.001$ for doubles. This shows clear evidence of pile up.

### 8.7 : My Observation is Piled Up! Now What?

There are a couple ways to deal with pile up. First, you can use event file filtering procedures to include only single pixel events `(PATTERN==0)`, as these events are less sensitive to pile up than other patterns.

You can also excise areas of high count rates, i.e., the boresight column and several columns to either side of it. (This is analogous to removing the inner-most regions of a source in Imaging data.) The spectrum can then be re-extracted and you can continue your analysis on the excised event file. As with Imaging data, it is recommended that you take an iterative approach: remove an inner region, extract a spectrum, check with epatplot, and repeat, each time removing a slightly larger region, until the model and observed pattern distributions agree.

To extract only the columns to either side of the boresight using the following expression when running `evselect`. All other inputs are the same as in §8.5.

<div class="alert alert-block alert-info">
    <b>Note:</b> We will not do the additional filtering for pile up here. We will just show the expression and inputs below. If you are only concerned with lower energies in the range of 500-2000 eV then pile up does not significantly affect this observation. But if you are interested in higher energies > 2000 eV, then you will need to correct for pile up. We recommend checking for pile up in the energy range you are interested in by using the <i>pileupnumberenergyrange</i> input for <i>epatplot</i>.
</div>

```python
expression = "'(FLAG==0)&&(PI in [500:15000])&&(RAWX in [3:5])&&!(RAWX in [29:45])'"

inargs = {}
inargs['table']           = basic_filter_file
inargs['withspectrumset'] = 'yes'
inargs['spectrumset']     = source_pi_file
inargs['energycolumn']    = 'PI'
inargs['spectralbinsize'] = '5'
inargs['withspecranges']  = 'yes'
inargs['specchannelmin']  = '0'
inargs['specchannelmax']  = '20479'
inargs['withfilteredset'] = 'yes'
inargs['filteredset']     = pn_spectra_file
inargs['expression']      = expression

w('evselect', inargs).run()
```

### 8.8 : Determine the Spectrum Extraction Areas

Now that we are confident that our spectrum is not piled up, we can continue by finding the source and background region areas. (This process is identical to that used for IMAGING data.) This is done with the task backscale, which takes into account any bad pixels or chip gaps, and writes the result into the BACKSCAL keyword of the spectrum table.

The inputs are:

    -spectrumset - (input) spectrum file
    -badpixlocation - (output) event file containing the bad pixels

In [ ]:
inargs = ['spectrumset={0}'.format(source_pi_file),
          'badpixlocation=pn_filt.fits']

w('backscale', inargs).run()

In [ ]:
inargs = ['spectrumset={0}'.format(bkg_pi_file),
          'badpixlocation=pn_filt.fits']

w('backscale', inargs).run()

### 8.9 : Create the Photon Redistribution Matrix (RMF) and Ancillary File (ARF)

Making the RMF and ARF for PN data in `TIMING` mode is exactly the same as in `IMAGING` mode, even if you had to excise piled up areas.

To make the RMF use `rmfgen`. The inputs are:

    -rmfset - output file
    -spectrumset - spectrum file

rmfgen rmfset=source_rmf_NoBore.fits spectrumset=source_pi_NoBore.fits

To make the ARF use `arfgen`. The inputs are:

    -arfset - output file
    -spectrumset - spectrum file
    -arfset - output file
    -detmaptype - origin of the detector map
    -withrmfset - use the RMF dataset to define the ARF energy grid?
    -rmfset - RMF file
    -badpixlocation - the file containing the bad pixel locations

In [ ]:
inargs = ['rmfset=source_rmf_NoBore.fits',
          'spectrumset={0}'.format(source_pi_file)]

w('rmfgen', inargs).run()

In [ ]:
inargs = ['arfset=source_arf_NoBore.fits',
          'spectrumset={0}'.format(source_pi_file),
          'detmaptype=psf',
          'withrmfset=yes',
          'rmfset=source_rmf_NoBore.fits',
          'badpixlocation=pn_filt.fits']

w('arfgen', inargs).run()

At this point, the spectrum is ready to be analyzed. How to fit the spectrum is explained in [Chapter 13 of the ABC Guide](https://heasarc.gsfc.nasa.gov/docs/xmm/abc/node15.html#Chap:epic-fit-xspec).